# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [3]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [4]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    
    source_text_sentences = [sentence for sentence in source_text.split('\n')]
    
    target_text_sentences = [sentence + ' <EOS>' for sentence in target_text.split('\n')]
    
    source_sentences_id = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_text_sentences]
    
    target_sentences_id = [[target_vocab_to_int[word] for word in sentence.split()] for sentence in target_text_sentences]
    
    return (source_sentences_id, target_sentences_id)
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [8]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, shape=(None, None), name='input')
    targets = tf.placeholder(tf.int32, shape=(None, None))
    learning_rate = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    target_seq_lenth = tf.placeholder(tf.int32, shape=[None], name= "target_sequence_length")
    max_target_len = tf.reduce_max(target_seq_lenth, name= 'max_target_len')
    source_seq_length = tf.placeholder(tf.int32, shape=[None], name= "source_sequence_length")
    
    return (inputs, targets, learning_rate, keep_prob, target_seq_lenth, max_target_len, source_seq_length)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [9]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    last_batch_id = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    result = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), last_batch_id], 1)
    return result

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [10]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    # Got help from udacity discussion forum
    # (https://discussions.udacity.com/t/help-me-for-the-encoding-layer-implementation/377704)
    
    
    n_stacked_layer = 2
    
    embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    
    def make_cell(rnn_size):
        
        cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
    
        drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
        return drop
    
    stacked_layer = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    output, state = tf.nn.dynamic_rnn(cell=stacked_layer, inputs=embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return output, state
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, helper, encoder_state, output_layer)
    f_output, _= tf.contrib.seq2seq.dynamic_decode(decoder,maximum_iterations=max_summary_length)
    return f_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [12]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    # got help from an online github repository and udacity discussion board
    
    start_ids = tf.tile([start_of_sequence_id], [batch_size])
    
    embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_ids, end_of_sequence_id)
    
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, embedding_helper, encoder_state, output_layer)
    
    f_output, _= tf.contrib.seq2seq.dynamic_decode(basic_decoder,maximum_iterations=max_target_sequence_length)
    
    return f_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [13]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm_drop
    
    stacking_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    dense_layer = Dense(target_vocab_size,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode") as scope:
        train_decoder_output = decoding_layer_train(
            encoder_state, stacking_lstm, dec_embed_input, 
            target_sequence_length, max_target_sequence_length, 
            dense_layer, keep_prob)
        
        scope.reuse_variables()
        
        inference_decoder_output = decoding_layer_infer(
            encoder_state, stacking_lstm, dec_embeddings, 
            target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], 
            max_target_sequence_length, target_vocab_size, 
            dense_layer, batch_size, keep_prob)
    
    return train_decoder_output, inference_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [14]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # ran into a couple of errors took help from
    # (https://discussions.udacity.com/t/def-seq2seq-model/277476)
    
    output, state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   enc_embedding_size)
    
    processed_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    training_decoder_output, inference_decoder_output = decoding_layer(processed_input, state,
                   target_sequence_length, max_target_sentence_length,
                   rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, dec_embedding_size)
    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [15]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 512
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 30
decoding_embedding_size = 30
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.75
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/1077 - Train Accuracy: 0.2379, Validation Accuracy: 0.3242, Loss: 5.0471
Epoch   0 Batch    2/1077 - Train Accuracy: 0.2463, Validation Accuracy: 0.3366, Loss: 5.5959
Epoch   0 Batch    3/1077 - Train Accuracy: 0.2766, Validation Accuracy: 0.3469, Loss: 4.2373
Epoch   0 Batch    4/1077 - Train Accuracy: 0.2750, Validation Accuracy: 0.3423, Loss: 4.4240
Epoch   0 Batch    5/1077 - Train Accuracy: 0.3031, Validation Accuracy: 0.3462, Loss: 4.0399
Epoch   0 Batch    6/1077 - Train Accuracy: 0.2930, Validation Accuracy: 0.3498, Loss: 3.7145
Epoch   0 Batch    7/1077 - Train Accuracy: 0.2742, Validation Accuracy: 0.3498, Loss: 3.7257
Epoch   0 Batch    8/1077 - Train Accuracy: 0.3113, Validation Accuracy: 0.3757, Loss: 3.6304
Epoch   0 Batch    9/1077 - Train Accuracy: 0.3180, Validation Accuracy: 0.3789, Loss: 3.3906
Epoch   0 Batch   10/1077 - Train Accuracy: 0.2932, Validation Accuracy: 0.3874, Loss: 3.5287
Epoch   0 Batch   11/1077 - Train Accuracy: 0.3449, Validati

Epoch   0 Batch   89/1077 - Train Accuracy: 0.4406, Validation Accuracy: 0.5082, Loss: 1.9289
Epoch   0 Batch   90/1077 - Train Accuracy: 0.4570, Validation Accuracy: 0.5110, Loss: 1.9368
Epoch   0 Batch   91/1077 - Train Accuracy: 0.5134, Validation Accuracy: 0.5167, Loss: 1.7014
Epoch   0 Batch   92/1077 - Train Accuracy: 0.4598, Validation Accuracy: 0.4915, Loss: 1.8048
Epoch   0 Batch   93/1077 - Train Accuracy: 0.4441, Validation Accuracy: 0.4979, Loss: 1.8452
Epoch   0 Batch   94/1077 - Train Accuracy: 0.4754, Validation Accuracy: 0.5053, Loss: 1.7879
Epoch   0 Batch   95/1077 - Train Accuracy: 0.4944, Validation Accuracy: 0.5210, Loss: 1.7785
Epoch   0 Batch   96/1077 - Train Accuracy: 0.4547, Validation Accuracy: 0.5021, Loss: 1.8010
Epoch   0 Batch   97/1077 - Train Accuracy: 0.4594, Validation Accuracy: 0.4968, Loss: 1.8007
Epoch   0 Batch   98/1077 - Train Accuracy: 0.4885, Validation Accuracy: 0.4893, Loss: 1.6915
Epoch   0 Batch   99/1077 - Train Accuracy: 0.4262, Validati

Epoch   0 Batch  177/1077 - Train Accuracy: 0.4811, Validation Accuracy: 0.5256, Loss: 1.4006
Epoch   0 Batch  178/1077 - Train Accuracy: 0.5051, Validation Accuracy: 0.5107, Loss: 1.3329
Epoch   0 Batch  179/1077 - Train Accuracy: 0.5173, Validation Accuracy: 0.5430, Loss: 1.3736
Epoch   0 Batch  180/1077 - Train Accuracy: 0.5094, Validation Accuracy: 0.5529, Loss: 1.3463
Epoch   0 Batch  181/1077 - Train Accuracy: 0.5129, Validation Accuracy: 0.5263, Loss: 1.3402
Epoch   0 Batch  182/1077 - Train Accuracy: 0.5130, Validation Accuracy: 0.5376, Loss: 1.3145
Epoch   0 Batch  183/1077 - Train Accuracy: 0.4902, Validation Accuracy: 0.5483, Loss: 1.3657
Epoch   0 Batch  184/1077 - Train Accuracy: 0.5395, Validation Accuracy: 0.5387, Loss: 1.2597
Epoch   0 Batch  185/1077 - Train Accuracy: 0.4918, Validation Accuracy: 0.5384, Loss: 1.3243
Epoch   0 Batch  186/1077 - Train Accuracy: 0.4860, Validation Accuracy: 0.5238, Loss: 1.3164
Epoch   0 Batch  187/1077 - Train Accuracy: 0.5094, Validati

Epoch   0 Batch  265/1077 - Train Accuracy: 0.5398, Validation Accuracy: 0.5678, Loss: 1.0240
Epoch   0 Batch  266/1077 - Train Accuracy: 0.5603, Validation Accuracy: 0.5781, Loss: 0.9775
Epoch   0 Batch  267/1077 - Train Accuracy: 0.5455, Validation Accuracy: 0.5799, Loss: 0.9226
Epoch   0 Batch  268/1077 - Train Accuracy: 0.5750, Validation Accuracy: 0.5799, Loss: 0.9995
Epoch   0 Batch  269/1077 - Train Accuracy: 0.5041, Validation Accuracy: 0.5554, Loss: 1.0708
Epoch   0 Batch  270/1077 - Train Accuracy: 0.5184, Validation Accuracy: 0.5586, Loss: 1.0493
Epoch   0 Batch  271/1077 - Train Accuracy: 0.5668, Validation Accuracy: 0.5746, Loss: 0.9837
Epoch   0 Batch  272/1077 - Train Accuracy: 0.5610, Validation Accuracy: 0.5732, Loss: 0.9613
Epoch   0 Batch  273/1077 - Train Accuracy: 0.5554, Validation Accuracy: 0.5895, Loss: 0.9766
Epoch   0 Batch  274/1077 - Train Accuracy: 0.5528, Validation Accuracy: 0.5689, Loss: 0.9650
Epoch   0 Batch  275/1077 - Train Accuracy: 0.5580, Validati

Epoch   0 Batch  353/1077 - Train Accuracy: 0.5436, Validation Accuracy: 0.5874, Loss: 0.9112
Epoch   0 Batch  354/1077 - Train Accuracy: 0.5809, Validation Accuracy: 0.6126, Loss: 0.8509
Epoch   0 Batch  355/1077 - Train Accuracy: 0.5908, Validation Accuracy: 0.6101, Loss: 0.7931
Epoch   0 Batch  356/1077 - Train Accuracy: 0.5742, Validation Accuracy: 0.5945, Loss: 0.8198
Epoch   0 Batch  357/1077 - Train Accuracy: 0.5949, Validation Accuracy: 0.5906, Loss: 0.7796
Epoch   0 Batch  358/1077 - Train Accuracy: 0.5374, Validation Accuracy: 0.5874, Loss: 0.8488
Epoch   0 Batch  359/1077 - Train Accuracy: 0.5715, Validation Accuracy: 0.5799, Loss: 0.8260
Epoch   0 Batch  360/1077 - Train Accuracy: 0.5668, Validation Accuracy: 0.5803, Loss: 0.8001
Epoch   0 Batch  361/1077 - Train Accuracy: 0.5958, Validation Accuracy: 0.5881, Loss: 0.8339
Epoch   0 Batch  362/1077 - Train Accuracy: 0.6019, Validation Accuracy: 0.5863, Loss: 0.8164
Epoch   0 Batch  363/1077 - Train Accuracy: 0.5414, Validati

...

Epoch   9 Batch  710/1077 - Train Accuracy: 0.9957, Validation Accuracy: 0.9730, Loss: 0.0055
Epoch   9 Batch  711/1077 - Train Accuracy: 0.9852, Validation Accuracy: 0.9780, Loss: 0.0160
Epoch   9 Batch  712/1077 - Train Accuracy: 0.9980, Validation Accuracy: 0.9830, Loss: 0.0063
Epoch   9 Batch  713/1077 - Train Accuracy: 0.9950, Validation Accuracy: 0.9830, Loss: 0.0066
Epoch   9 Batch  714/1077 - Train Accuracy: 0.9914, Validation Accuracy: 0.9830, Loss: 0.0116
Epoch   9 Batch  715/1077 - Train Accuracy: 0.9832, Validation Accuracy: 0.9830, Loss: 0.0114
Epoch   9 Batch  716/1077 - Train Accuracy: 0.9895, Validation Accuracy: 0.9798, Loss: 0.0065
Epoch   9 Batch  717/1077 - Train Accuracy: 0.9984, Validation Accuracy: 0.9798, Loss: 0.0049
Epoch   9 Batch  718/1077 - Train Accuracy: 0.9918, Validation Accuracy: 0.9748, Loss: 0.0075
Epoch   9 Batch  719/1077 - Train Accuracy: 0.9989, Validation Accuracy: 0.9751, Loss: 0.0070
Epoch   9 Batch  720/1077 - Train Accuracy: 0.9934, Validati

Epoch   9 Batch  798/1077 - Train Accuracy: 0.9883, Validation Accuracy: 0.9744, Loss: 0.0117
Epoch   9 Batch  799/1077 - Train Accuracy: 0.9844, Validation Accuracy: 0.9744, Loss: 0.0119
Epoch   9 Batch  800/1077 - Train Accuracy: 0.9961, Validation Accuracy: 0.9744, Loss: 0.0067
Epoch   9 Batch  801/1077 - Train Accuracy: 0.9773, Validation Accuracy: 0.9744, Loss: 0.0096
Epoch   9 Batch  802/1077 - Train Accuracy: 0.9799, Validation Accuracy: 0.9744, Loss: 0.0063
Epoch   9 Batch  803/1077 - Train Accuracy: 0.9965, Validation Accuracy: 0.9744, Loss: 0.0059
Epoch   9 Batch  804/1077 - Train Accuracy: 1.0000, Validation Accuracy: 0.9794, Loss: 0.0050
Epoch   9 Batch  805/1077 - Train Accuracy: 0.9945, Validation Accuracy: 0.9794, Loss: 0.0084
Epoch   9 Batch  806/1077 - Train Accuracy: 0.9926, Validation Accuracy: 0.9794, Loss: 0.0076
Epoch   9 Batch  807/1077 - Train Accuracy: 0.9953, Validation Accuracy: 0.9744, Loss: 0.0026
Epoch   9 Batch  808/1077 - Train Accuracy: 0.9781, Validati

Epoch   9 Batch  886/1077 - Train Accuracy: 0.9902, Validation Accuracy: 0.9819, Loss: 0.0114
Epoch   9 Batch  887/1077 - Train Accuracy: 0.9883, Validation Accuracy: 0.9819, Loss: 0.0127
Epoch   9 Batch  888/1077 - Train Accuracy: 0.9881, Validation Accuracy: 0.9773, Loss: 0.0094
Epoch   9 Batch  889/1077 - Train Accuracy: 0.9844, Validation Accuracy: 0.9769, Loss: 0.0070
Epoch   9 Batch  890/1077 - Train Accuracy: 0.9996, Validation Accuracy: 0.9769, Loss: 0.0099
Epoch   9 Batch  891/1077 - Train Accuracy: 0.9988, Validation Accuracy: 0.9769, Loss: 0.0057
Epoch   9 Batch  892/1077 - Train Accuracy: 0.9871, Validation Accuracy: 0.9769, Loss: 0.0125
Epoch   9 Batch  893/1077 - Train Accuracy: 0.9848, Validation Accuracy: 0.9769, Loss: 0.0097
Epoch   9 Batch  894/1077 - Train Accuracy: 0.9952, Validation Accuracy: 0.9769, Loss: 0.0043
Epoch   9 Batch  895/1077 - Train Accuracy: 0.9945, Validation Accuracy: 0.9716, Loss: 0.0093
Epoch   9 Batch  896/1077 - Train Accuracy: 0.9831, Validati

Epoch   9 Batch  974/1077 - Train Accuracy: 0.9801, Validation Accuracy: 0.9759, Loss: 0.0080
Epoch   9 Batch  975/1077 - Train Accuracy: 0.9922, Validation Accuracy: 0.9759, Loss: 0.0089
Epoch   9 Batch  976/1077 - Train Accuracy: 0.9801, Validation Accuracy: 0.9808, Loss: 0.0088
Epoch   9 Batch  977/1077 - Train Accuracy: 0.9906, Validation Accuracy: 0.9808, Loss: 0.0059
Epoch   9 Batch  978/1077 - Train Accuracy: 0.9957, Validation Accuracy: 0.9755, Loss: 0.0085
Epoch   9 Batch  979/1077 - Train Accuracy: 0.9807, Validation Accuracy: 0.9755, Loss: 0.0098
Epoch   9 Batch  980/1077 - Train Accuracy: 0.9809, Validation Accuracy: 0.9755, Loss: 0.0079
Epoch   9 Batch  981/1077 - Train Accuracy: 0.9871, Validation Accuracy: 0.9755, Loss: 0.0077
Epoch   9 Batch  982/1077 - Train Accuracy: 0.9926, Validation Accuracy: 0.9755, Loss: 0.0070
Epoch   9 Batch  983/1077 - Train Accuracy: 0.9885, Validation Accuracy: 0.9755, Loss: 0.0065
Epoch   9 Batch  984/1077 - Train Accuracy: 0.9738, Validati

Epoch   9 Batch 1062/1077 - Train Accuracy: 0.9996, Validation Accuracy: 0.9883, Loss: 0.0035
Epoch   9 Batch 1063/1077 - Train Accuracy: 0.9930, Validation Accuracy: 0.9883, Loss: 0.0105
Epoch   9 Batch 1064/1077 - Train Accuracy: 0.9941, Validation Accuracy: 0.9883, Loss: 0.0084
Epoch   9 Batch 1065/1077 - Train Accuracy: 0.9758, Validation Accuracy: 0.9883, Loss: 0.0088
Epoch   9 Batch 1066/1077 - Train Accuracy: 0.9914, Validation Accuracy: 0.9883, Loss: 0.0166
Epoch   9 Batch 1067/1077 - Train Accuracy: 0.9809, Validation Accuracy: 0.9858, Loss: 0.0111
Epoch   9 Batch 1068/1077 - Train Accuracy: 0.9887, Validation Accuracy: 0.9858, Loss: 0.0079
Epoch   9 Batch 1069/1077 - Train Accuracy: 0.9937, Validation Accuracy: 0.9854, Loss: 0.0080
Epoch   9 Batch 1070/1077 - Train Accuracy: 0.9973, Validation Accuracy: 0.9854, Loss: 0.0045
Epoch   9 Batch 1071/1077 - Train Accuracy: 0.9879, Validation Accuracy: 0.9854, Loss: 0.0107
Epoch   9 Batch 1072/1077 - Train Accuracy: 0.9933, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [21]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    words = sentence.lower().split()
    word_id_list = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in words]
    return word_id_list


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [22]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [225, 34, 179, 195, 163, 90, 13]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [108, 9, 130, 298, 61, 175, 276, 1]
  French Words: il vu un vieux camion jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.